In [ ]:
%aiida
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

In [ ]:
from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.array import ArrayData
from aiida.orm.data.base import Int, Str, Float, Bool
from aiida.orm.data.remote import RemoteData
from aiida.work import workfunction
from aiida.work.process import WorkCalculation
from aiida.work.run import submit
from aiida_cp2k.calculations import Cp2kCalculation
from aiida.orm import Code, Computer
from aiida.orm.querybuilder import QueryBuilder

import ase
import ase.io
import numpy as np
import nglview
from copy import deepcopy
from pprint import pprint

import ipywidgets as ipw
from IPython.display import display, clear_output, HTML

from apps.scanning_probe.pdos.pdos_workchain import PdosWorkChain

# Select structure

In [ ]:
from apps.surfaces.structure_browser import StructureBrowser

def on_struct_change(c):
    update_view()
    guess_molecule()

    
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')
viewer = nglview.NGLWidget()
clear_output()

display(ipw.VBox([struct_browser, viewer]))

def update_view():
    # remove old components
    if hasattr(viewer, "component_0"):
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
        
    structure = struct_browser.results.value
    if not structure:
        return
    
    atoms = structure.get_ase()
    #print("CELL:", atoms.cell)
    # add new component
    viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
    viewer.add_unitcell()
    viewer.center()
    
    mol_inds = np.arange(1, slab_start.value)-1
    
    viewer.add_representation('ball+stick', selection=mol_inds, color='green', aspectRatio=4.0, opacity=0.4)
    

In [ ]:
def get_opt_wfn_file(structure):
    """Tries to find the path to the RESTART.wfn file, if it exists."""
    
    cp2k_calc = structure.get_inputs()[0]
    comp = cp2k_calc.get_computer()
    folder = cp2k_calc.out.remote_folder.get_remote_path()
    
    return comp, folder

# Select molecule

The atoms of the molecule need to be ordered in front of the data structure.

In [ ]:
def guess_molecule():
    try:
        ase_struct = struct_browser.results.value.get_ase()
        first_slab_atom = np.argwhere( (ase_struct.numbers == 29) |
                                       (ase_struct.numbers == 47) |
                                       (ase_struct.numbers == 79)
                                     )[0, 0]
        slab_start.value=first_slab_atom+1
    except:
        print("Unable to automatically find slab and molecule")

def get_mol_ase():
    mol_inds = np.arange(1, slab_start.value)-1
    ase_struct = struct_browser.results.value.get_ase()
    return ase_struct[mol_inds]
    

style = {'description_width': '120px'}
layout = {'width': '60%'}

slab_start = ipw.IntText(description='Slab start index',
                         value=1,
                         min=1,
                         style=style, layout={'width': '50%'})

highlight_btn = ipw.Button(description='highlight',
                            layout={'width': '10%'})

highlight_btn.on_click(lambda b: update_view())

display(ipw.HBox([slab_start, highlight_btn]))

# Select computer and codes

In [ ]:
qb = QueryBuilder()
qb.append(Computer, filters={'enabled': True})
computer_names = [comp[0].name for comp in qb.all()]

style = {'description_width': '140px'}
layout = {'width': '70%'}
drop_computer = ipw.Dropdown(description="Computer",
                             options=computer_names)

def comp_plugin_codes(computer_name, plugin_name):
    qb = QueryBuilder()
    qb.append(Computer, filters={'enabled': True}, project='name', tag='computer')
    qb.append(Code, project='*', has_computer='computer', filters={'attributes.input_plugin': plugin_name})
    qb.order_by({Code: {'id': 'desc'}})
    codes = qb.all()
    sel_codes = []
    for code in codes:
        if code[0] == computer_name:
            sel_codes.append(code[1])
    return sel_codes

def on_computer_change(c):
    global cp2k_codes, eval_morb_codes, overlap_codes
    cp2k_codes = comp_plugin_codes(drop_computer.value, 'cp2k')
    eval_morb_codes = comp_plugin_codes(drop_computer.value, 'spm.eval_morbs')
    overlap_codes = comp_plugin_codes(drop_computer.value, 'spm.overlap')
    
    drop_cp2k.options = [c.label for c in cp2k_codes]
    drop_eval.options = [c.label for c in eval_morb_codes]
    drop_over.options = [c.label for c in overlap_codes]
    
    
drop_computer.observe(on_computer_change)

drop_cp2k = ipw.Dropdown(description="Cp2k code")

drop_eval = ipw.Dropdown(description="Eval code")

drop_over = ipw.Dropdown(description="Overlap code")

on_computer_change(0)

elpa_check = ipw.Checkbox(
    value=True,
    description='use ELPA',
    disabled=False
)

display(drop_computer, drop_cp2k, drop_eval, drop_over, elpa_check)

# Orbital overlap parameters

In [ ]:
layout_small = {'width': '30%'}

num_homo_inttext = ipw.IntText(
                    value=4,
                    description="num gas HOMO",
                    style=style, layout=layout_small)
num_lumo_inttext = ipw.IntText(
                    value=4,
                    description="num gas LUMO",
                    style=style, layout=layout_small)

elim_inttext = ipw.IntText(
                    value=4,
                    description="num gas LUMO",
                    style=style, layout=layout_small)

emin_floattext = ipw.BoundedFloatText(
                        description='Slab system Emin (eV)',
                        min=-3.0,
                        max=-0.1,
                        step=0.1,
                        value=-2.0,
                        style=style, layout=layout_small)

emax_floattext = ipw.BoundedFloatText(
                        description='Slab system Emax (eV)',
                        min=0.1,
                        max=3.0,
                        step=0.1,
                        value=2.0,
                        readout_format='%.2f',
                        style=style, layout=layout_small)

display(num_homo_inttext, num_lumo_inttext, emin_floattext, emax_floattext)

In [ ]:
def find_struct_wf(structure):
    selected_comp = qb.all()[drop_computer.index][0]
    
    # check stm
    extras = structure.get_extras()
    for ex_k in extras.keys():
        if ex_k.startswith('stm'):
            stm_workchain = load_node(extras[ex_k])
            cp2k_scf_calc = stm_workchain.get_outputs()[0]
            if cp2k_scf_calc.get_computer().hostname == selected_comp.hostname:
                wfn_path = cp2k_scf_calc.out.remote_folder.get_remote_path() + "/aiida-RESTART.wfn"
                # check if it exists
                file_exists = ! ssh {selected_comp.hostname} "if [ -f {wfn_path} ]; then echo 1 ; else echo 0 ; fi"
                if file_exists[0]:
                    print("Found .wfn from %s"%ex_k)
                    return wfn_path
    
    # check geo opt
    geo_opt_calc = structure.get_inputs()[0]
    geo_comp = geo_opt_calc.get_computer()
    if geo_comp.hostname == selected_comp.hostname:
        wfn_path = cp2k_calc.out.remote_folder.get_remote_path() + "/aiida-RESTART.wfn"
        # check if it exists
        file_exists = ! ssh {selected_comp.hostname} "if [ -f {wfn_path} ]; then echo 1 ; else echo 0 ; fi"
        if file_exists[0]:
            print("Found .wfn from geo_opt")
            return wfn_path
    return ""

def on_submit(b):
    with submit_out:
        clear_output()
        if not struct_browser.results.value:
            print("Please select a structure.")
            return
        if not drop_computer.value:
            print("Please select a computer.")
            return
        
        if not drop_cp2k.value or not drop_eval.value or not drop_over.value:
            print("Please select all the codes.")
            return
        
        parent_dir = "parent_calc/"

        eval_slab_params = ParameterData(dict={
            '--cp2k_input':  parent_dir+'aiida.inp',
            '--basis_file':  parent_dir+'BASIS_MOLOPT',
            '--xyz_file':    parent_dir+'geom.xyz',
            '--wfn_file':    parent_dir+'aiida-RESTART.wfn',
            '--output_file': 'morbs_dx0.2',
            '--emin':        str(emin_floattext.value),
            '--emax':        str(emax_floattext.value),
            '--eval_region': ['G', 'G', 'G', 'G', 'b-4.0_C', 't2.0'],
            '--dx':          '0.2',
            '--eval_cutoff': '14.0'
        })
        
        eval_mol_params = ParameterData(dict={
            '--cp2k_input':  parent_dir+'aiida.inp',
            '--basis_file':  parent_dir+'BASIS_MOLOPT',
            '--xyz_file':    parent_dir+'geom.xyz',
            '--wfn_file':    parent_dir+'aiida-RESTART.wfn',
            '--output_file': 'morbs_dx0.2',
            '--emin':        str(num_homo_inttext.value*0.8),
            '--emax':        str(num_lumo_inttext.value*0.8),
            '--eval_region': ['G', 'G', 'G', 'G', 'b-4.0_C', 't2.0'],
            '--dx':          '0.2',
            '--eval_cutoff': '14.0'
        })

        overlap_params = ParameterData(dict={
            '--npz_file1':         'parent_mol_folder/morbs_dx0.2.npz',
            '--npz_file2':         'parent_slab_folder/morbs_dx0.2.npz',
            '--n_homo':            str(num_homo_inttext.value),
            '--n_lumo':            str(num_lumo_inttext.value)
        })
        
        cp2k_code = cp2k_codes[drop_cp2k.index]
        eval_orbs_code = eval_morb_codes[drop_eval.index]
        overlap_code = overlap_codes[drop_over.index]
        
        struct = struct_browser.results.value
        
        ## Try to access the restart-wfn file ##
        try:
            wfn_file_path = find_struct_wf(struct)
        except:
            wfn_file_path = ""
        if wfn_file_path == "":
            print("Didn't find any accessible .wfn file.")
        
        mol_structure = StructureData(ase=get_mol_ase())
        mol_structure.description = "mol from slab PK%d" %  struct.pk
        
        outputs = submit(PdosWorkChain,
                 cp2k_code=cp2k_code,
                 slabsys_structure=struct,
                 mol_structure=mol_structure,
                 first_slab_ind=Int(slab_start.value),
                 wfn_file_path=Str(wfn_file_path),
                 elpa_switch=Bool(elpa_check.value),
                 eval_orbs_code=eval_orbs_code,
                 eval_slab_params=eval_slab_params,
                 eval_mol_params=eval_mol_params,
                 overlap_code=overlap_code,
                 overlap_params=overlap_params
                )
                
        print(outputs)

btn_submit = ipw.Button(description="Submit")
btn_submit.on_click(on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)